## Import necessary python modules

In [1]:
import os
from pathlib import Path
import napari
import trackpy as tp
import tifffile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tempfile import TemporaryFile
import bigfish
import bigfish.plot as plot
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from random import sample 
pd.set_option('display.max_rows', 2000)
from copy import deepcopy
from dask.array.image import imread as imr
from bigfish.detection.utils import get_object_radius_pixel

from pathlib import Path
from napari.layers import Image
from magicgui import magicgui, magic_factory, widgets
from magicgui.widgets import Table
import bigfish.segmentation as segmentation
from skimage.segmentation import find_boundaries
from skimage.morphology.selem import disk
from napari.types import ImageData, LabelsData
from skimage.measure import label, regionprops

C:\Users\uid-1204\AppData\Local\anaconda3\envs\bigfishLive\lib\site-packages\bigfish\segmentation\nuc_segmentation.py:16: FutureWarning: The `skimage.morphology.selem` module is deprecated and will be removed in scikit-image 1.0 (`skimage.morphology.selem` has been moved to `skimage.morphology.footprints`).
  from skimage.morphology.selem import disk


## Specify voxel and object size


In [2]:
voxelRadius = (600, 121, 121)
objectRadius = (600, 150, 150)

In [3]:
from scipy.optimize import curve_fit

def bi_exp(x, a, b, c, d):
    return (a * np.exp(-b * x)) + (c * np.exp(-d * x))

def trip_exp(x, a, b, c, d, e, f):
    return ((a * np.exp(-b * x)) + (c * np.exp(-d * x)) + (e * np.exp(-f * x)))

def getBleachCorrected(stackCell, model='bi'):
    axes = tuple([i for i in range(len(stackCell.shape))])
    I_mean = np.mean(stackCell, axis=axes[1:])
    timePoints = np.arange(stackCell.shape[0])
    
    if model=='bi':
        coeffsExp, _ = curve_fit(bi_exp, timePoints, I_mean, maxfev=50000)
        f_ = np.vectorize(bi_exp)(timePoints, *coeffsExp)
    elif model=='tri':
        coeffsExp, _ = curve_fit(trip_exp, timePoints, I_mean, maxfev=50000)
        f_ = np.vectorize(trip_exp)(timePoints, *coeffsExp)
    
    
    f = f_ / np.max(f_)
    f = f.reshape(-1, 1, 1, 1)
    imagesCorrected = (stackCell / f).astype(np.uint16)

    # calculate r squared
    residuals = I_mean - f_
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((I_mean - np.mean(I_mean)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    r_squared_exp = np.array(r_squared)
    return imagesCorrected, r_squared_exp, I_mean

def getImagesAndSpotList(sequenceCell, selectedThreshold, voxelRadius, objectRadius, sampling=10):
    images=[]    
    spots_list=[]
    MaxTimePoint = sequenceCell.shape[0]

    spot_radius_px = detection.get_object_radius_pixel(
        voxel_size_nm=voxelRadius, 
        object_radius_nm=objectRadius, 
        ndim=3)

    for t in range(1,MaxTimePoint,sampling):
        rna = np.array(sequenceCell[t])
        images.append(rna)

        # LoG filter
        rna_log = stack.log_filter(rna, sigma=spot_radius_px)

        # local maximum detection
        mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)

        # thresholding
        threshold = detection.automated_threshold_setting(rna_log, mask)
        spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold))
        spots_list.append(spots_)
    n=len(images)
    print("Total number of images : "+str(n))
    return images, spots_list, n

def choose_home_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askdirectory(initialdir= "/", title='Please select a directory')  # Open file dialog

    root.destroy()  # Close the tkinter window
    return file_path


def findFitstTxFrame(blurClusters):
    for i in range(len(blurClusters)):  
        if blurClusters[i].size!=0:
            return i

def findSpotBrightness2D(spotList, blurImage):
    meanBrightness = []
    for hh in range(len(spotList)):
#         z = spotList[hh,0]
        y = spotList[hh,0]
        x = spotList[hh,1]
        meanBrightness.append(np.sum(np.array(blurImage[y-3:y+4,x-3:x+4])))
    return meanBrightness

from runBigfishDetection import getSpotAndClusters
def mergeTxSites(clusterFrames):
    newCluster = np.zeros((1,5))
    newCluster[0,0:3]=clusterFrames[np.argmax(clusterFrames[:,3]), 0:3]
    newCluster[0,3]= np.sum(clusterFrames[:,3])
    return newCluster


def df2List(potentialTxs):
    newClusterFrame=[]
    
    for i in range(len(potentialTxs)):
        if potentialTxs.iloc[i,1:].sum()!=0:
            newClusterFrame.append(np.array(potentialTxs.iloc[i,1:]).reshape((1, 5)))
        else:
            newClusterFrame.append(np.array([], dtype=np.int64).reshape((0, 5)))
    return newClusterFrame
def findSpotBrightness(spotList, sequenceCell):
    meanBrightness = []
    for hh in range(len(spotList)):
        z = spotList[hh,0]
        y = spotList[hh,1]
        x = spotList[hh,2]
        meanBrightness.append(np.sum(np.array(sequenceCell[z-3:z+4,y-3:y+4,x-3:x+4])))
    return meanBrightness

def findMissingTxSite(selectedThreshold, sequenceCell, reference_spot,cellNumber,Tx_label_clean, t, lastLocation, bETA, gAMA=5, numCs=2, rCls=600, vR=(600,121,121), oR=(400,202,202), reorderyn = False):
    spots_T, clusters_T, _ = getSpotAndClusters(sequenceCell,
                                             reference_spot, 
                                             cellnumber=cellNumber, 
                                             startTime=t,
                                             stopTime=t+1, 
                                             thresholdManual=selectedThreshold, 
                                             beta=bETA, 
                                             gamma=gAMA,
                                             numberOfSpots=numCs,
                                             radiusCluster=rCls, 
                                             voxelSize=vR, 
                                             objectSize=oR,
                                             reorder=reorderyn,
                                             extensionMov='.tif',
                                             showProgress=False)
#     print(spots_T, clusters_T)
#     print('last loc: ', lastLocation)
    if len(clusters_T[0])>3*len(spots_T[0]):
        print(len(spots_T[0]))
        print(len(clusters_T[0]))
        print('beta very low! Increase the beta!')
        
    mask_in_frames = Tx_label_clean[clusters_T[0][:,1], clusters_T[0][:,2]]
#     print(clusters_T)
    mask_in_frames = Tx_label_clean[clusters_T[0][:,1], clusters_T[0][:,2]]
    missingTx0 = clusters_T[0][mask_in_frames]

#     print(missingTx0)
    if missingTx0.size!=0:
        idxs = np.where(np.sum(np.abs(missingTx0[:,0:3] - lastLocation), axis=1)<=10)[0]
        if idxs.size!=0:
            missingTx = mergeTxSites(missingTx0[idxs,:])
            return missingTx
        else:
            print('not found (not close to last location)')
            return missingTx0[idxs,:]
    else:
        print('not found (not in mask)')
        return missingTx0

def df2ClusterData(potentialTxs):
    newClusterFrame=[]
    
    for i in range(MaxTimePoint):
        if np.sum(potentialTxs.loc[potentialTxs['frame']==i][['z','y', 'x','mrna']]).sum()!=0:
            clustersFound = np.array(potentialTxs.loc[potentialTxs['frame']==i][['z','y', 'x','mrna', 'cluster_id']])
            lsize=sum(np.sum(clustersFound[:,:-1], axis=1).astype(bool))
            newClusterFrame.append(clustersFound[np.sum(clustersFound[:,:-1], axis=1).astype(bool)].reshape((lsize, 5)))
        else:
            newClusterFrame.append(np.array([], dtype=np.int64).reshape((0, 5)))
    return newClusterFrame

def makeTrackData(potentialTxsFinal):
    newClusterFrame = df2ClusterData(potentialTxsFinal)
    columnsid=['frame']
    for clidu in np.unique(potentialTxsFinal.cluster_id):
        if clidu ==0:
            Tx1 = potentialTxsFinal[potentialTxsFinal['cluster_id']==0]
            tableToDisplay = pd.DataFrame(Tx1.loc[:,['frame', 'mrna']].astype(int))
            columnsid.append('mrna_'+str(clidu))
        else:
            Tx2 = potentialTxsFinal[potentialTxsFinal['cluster_id']==clidu]
            tableToDisplay = pd.concat([tableToDisplay , Tx2.loc[:,['mrna']].astype(int)], axis=1)
            columnsid.append('mrna_'+str(clidu))
    tableToDisplay.columns = columnsid
    columnHd = tableToDisplay.columns[1:]
    dftracksFinal = pd.concat([potentialTxsFinal['cluster_id'],potentialTxsFinal['frame'],potentialTxsFinal.loc[:,['y','x']]], axis=1)
    dd=dftracksFinal.sort_values(by=['cluster_id', 'frame'])
    tracks_data = np.asarray(dd)
    dftracksFinal = dftracksFinal[np.sum(dftracksFinal.iloc[:,2:],axis=1)!=0]
    return columnHd, newClusterFrame, tableToDisplay, dftracksFinal

In [4]:
def setProbability(part1Test):
    probTx = np.zeros((900,1))
    windowSize=5
    start=windowSize//2
    stop=MaxTimePoint-start

    for t in part1Test.frame.values:
        for ii in np.arange(-(windowSize-1)//2,(windowSize+1)//2,1):
            probTx[t+ii] = probTx[t+ii]+(1/9)
    return probTx

def findDetectedTxs(clsFrm, part1Test, MaxTimePoint, probTx):
    potentialTxs = pd.DataFrame(np.zeros([MaxTimePoint, 1+3+1]), columns=['frame', 'z', 'y', 'x', 'mrna'])
    potentialTxs['frame']= np.arange(0,MaxTimePoint,1)
    lastLocation = clsFrm[findFitstTxFrame(clsFrm)][:,1:3]
    for frameNumber in range(0,MaxTimePoint):
        clsFrmSrtd=clsFrm[frameNumber][np.argsort(clsFrm[frameNumber][:,0]),:]
        if len(clsFrm[frameNumber])>2:
    #         print('more than two found at', frameNumber)
            idx = np.where(np.sum(np.power(clsFrmSrtd[:,1:3]-np.array(lastLocation),2), axis=1)<=60)[0]
            if idx.size!=0:
                validIdxs=np.unique(idx)
                clsFrm[frameNumber] = mergeTxSites(clsFrmSrtd[validIdxs,:])
    #             print('merged')
            elif idx.size==0:
                t = part1Test.frame.values[np.argmin(abs(part1Test.frame.values-frameNumber))]
                if np.abs(t-frameNumber)<=5:        
                    particleCoordinates = np.asarray(part1Test[part1Test['frame']==t])[0][:2]
                    print('possible error, looking for hct at t', t)
                    idxnew = np.where(np.sum(np.power(clsFrmSrtd[:,1:3]-particleCoordinates,2), axis=1)<=70)[0]
                    if idxnew.size!=0:
                        clsFrm[frameNumber] = mergeTxSites(clsFrmSrtd[idxnew,:])
                        print('new tx found', frameNumber)
                        potentialTxs.loc[potentialTxs['frame']==frameNumber,['z', 'y', 'x', 'mrna']] = clsFrm[frameNumber][0][0:4]
                        lastLocation = np.array(potentialTxs.loc[potentialTxs['frame']==frameNumber,['y', 'x']])
                    if idxnew.size==0:
                        if probTx[frameNumber]>0.7:
                            idxnew2 = np.where(np.sum(np.abs(clsFrmSrtd[:,1:3]-particleCoordinates), axis=1)<=70)[0]
                            clsFrm[frameNumber] = mergeTxSites(clsFrmSrtd[idxnew2,:])
                            print('new tx added regardless since high prob', frameNumber)
                            potentialTxs.loc[potentialTxs['frame']==frameNumber,['z', 'y', 'x', 'mrna']] = clsFrm[frameNumber][0][0:4]
                            lastLocation = np.array(potentialTxs.loc[potentialTxs['frame']==frameNumber,['y', 'x']])

        if len(clsFrm[frameNumber])==1:
    #         print('only one tx found', frameNumber)
            potentialTxs.loc[potentialTxs['frame']==frameNumber,['z', 'y', 'x', 'mrna']] = clsFrm[frameNumber][0][0:4]
            lastLocation = np.array(potentialTxs.loc[potentialTxs['frame']==frameNumber,['y', 'x']])
    #         print('added')

        elif len(clsFrm[frameNumber])==2:
    #         print('frame :',frameNumber, ' lastloc ',lastLocation)
            idx = np.where(np.sum(np.power(clsFrmSrtd[:,1:3]-np.array(lastLocation),2), axis=1)<=60)[0]
    #         print('2 clusters found', frameNumber, 'idxs :' ,idx)
            if idx.size!=0:
                validIdxs=np.unique(idx)
    #             print('merging sites')
                clsFrm[frameNumber] = mergeTxSites(clsFrmSrtd[validIdxs,:])
                potentialTxs.loc[potentialTxs['frame']==frameNumber, ['z', 'y', 'x']]=clsFrm[frameNumber][:, 0:3]
                potentialTxs.loc[potentialTxs['frame']==frameNumber, ['mrna']] = clsFrm[frameNumber][:,3]
                lastLocation = np.array(potentialTxs.loc[potentialTxs['frame']==frameNumber,['y', 'x']])
            elif idx.size==0:
                t = part1Test.frame.values[np.argmin(abs(part1Test.frame.values-frameNumber))]
                if np.abs(t-frameNumber)<=5:        
                    particleCoordinates = np.asarray(part1Test[part1Test['frame']==t])[0][:2]
                    print('possible error, looking for hct at t', t)
                    idxnew = np.where(np.sum(np.power(clsFrmSrtd[:,1:3]-particleCoordinates,2), axis=1)<=25)[0]
                    if idxnew.size!=0:
                        clsFrm[frameNumber] = mergeTxSites(clsFrmSrtd[idxnew,:])
                        print('new tx found', frameNumber)
                        potentialTxs.loc[potentialTxs['frame']==frameNumber,['z', 'y', 'x', 'mrna']] = clsFrm[frameNumber][0][0:4]
                        lastLocation = np.array(potentialTxs.loc[potentialTxs['frame']==frameNumber,['y', 'x']])
                    if idxnew.size==0:
                         if probTx[frameNumber]>0.7:
                            idxnew2 = np.where(np.sum(np.abs(clsFrmSrtd[:,1:3]-particleCoordinates), axis=1)<=100)[0]

                            clsFrm[frameNumber] = mergeTxSites(clsFrmSrtd[idxnew2,:])
                            print('new tx added regardless since high prob', frameNumber)
                            potentialTxs.loc[potentialTxs['frame']==frameNumber,['z', 'y', 'x', 'mrna']] = clsFrm[frameNumber][0][0:4]
                            lastLocation = np.array(potentialTxs.loc[potentialTxs['frame']==frameNumber,['y', 'x']])
    return potentialTxs



In [5]:
def modifyFinalTrackDataFrame(potentialTxs, potentialTxsFinal,total_number_of_tx_site):
    #--------------------------------- Make dataframe
    if np.size(potentialTxsFinal)==0:
        if 'cluster_id' not in potentialTxs.columns:
            potentialTxs['cluster_id']=0
        potentialTxsFinal = deepcopy(potentialTxs)    
    elif len(np.unique(potentialTxsFinal.cluster_id)) < total_number_of_tx_site:
        new_cluster_id = np.unique(potentialTxsFinal['cluster_id'])[-1]+1
        potentialTxs['cluster_id']=new_cluster_id
        potentialTxsFinal = pd.concat([potentialTxsFinal, potentialTxs])
    elif len(np.unique(potentialTxsFinal.cluster_id)) == total_number_of_tx_site:
        chngid = int(input('Enter track to replace : '))
        potentialTxs['cluster_id']=chngid
        potentialTxsFinal.loc[potentialTxsFinal['cluster_id']==chngid]=potentialTxs
    return potentialTxs, potentialTxsFinal

def findMissingTranscriptionSitesFromTracks(potentialTxs, part1Test,spotsFrame,voxelRadius, objectRadius,selectedThreshold, sequenceCell, reference_spot,cellNumber,Tx_mask_new, BETA, gM,nP):
    #---------------------- Find missing Txs
    missingTXFrames = []
    for frameNumber in range(len(potentialTxs)):  
        t = part1Test.frame.values[np.argmin(abs(part1Test.frame.values-frameNumber))]
        if np.abs(t-frameNumber)<=5:        
            particleCoordinates = np.asarray(part1Test[part1Test['frame']==t])[0][:2]
            if potentialTxs.iloc[frameNumber,2:4].sum()!=0:           
                if np.sum(np.power(potentialTxs.iloc[frameNumber,2:4]-particleCoordinates,2))>25:
                    brightest = spotsFrame[frameNumber][np.argsort(findSpotBrightness(spotsFrame[frameNumber][:,:], sequenceCell[frameNumber]))[-1]]
                    if np.sum(np.abs(brightest-clsFrm[frameNumber][:,0:3]))>3:
                        print('error in frame ', frameNumber, t)

            elif potentialTxs.iloc[frameNumber,2:4].sum()==0:
                if probTx[frameNumber]>0.6:
                    if spotsFrame[frameNumber].size!=0:                
                        brightest = spotsFrame[frameNumber][np.argsort(findSpotBrightness2D(spotsFrame[frameNumber][:,1:], mipSequenceCell[frameNumber]))[-1]]
                        if clsFrm[frameNumber-1][:,0:3].size!=0:
                            if np.sum(np.abs(brightest-clsFrm[frameNumber-1][:,0:3]))<4:
                                print('missing Tx at :', frameNumber)
                                missingTXFrames.append(frameNumber)
                        elif clsFrm[frameNumber+1][:,0:3].size!=0:
                            if np.sum(np.abs(brightest-clsFrm[frameNumber+1][:,0:3]))<4:
                                print('missing Tx at :', frameNumber)
                                missingTXFrames.append(frameNumber)
    
    print('Found possible missing transcription sites: ', len(missingTXFrames))

    for ff in missingTXFrames:
        print(ff)
        lst = np.array(potentialTxs[potentialTxs['frame']==ff-1])[:,1:4]
        if np.sum(lst)!=0:
            lastLocation = lst
        else:
            lastLocation = np.array(potentialTxs[potentialTxs['frame']==ff+1])[:,1:4]
        newtxs = findMissingTxSite(selectedThreshold, sequenceCell, reference_spot,cellNumber, Tx_mask_new, ff, lastLocation, bETA=BETA, gAMA=gM, numCs=nP, vR=voxelRadius, oR=objectRadius,reorderyn = False)
        if newtxs.size!=0:
            potentialTxs.loc[potentialTxs['frame']==ff,['z', 'y', 'x', 'mrna']] = newtxs[:,:4]
    return potentialTxs

def plotCoordinatesFound(potentialTxs, particle, plotFileName):
    plt.rcParams.update({'font.size': 18})
    plt.figure(figsize=(20,8))
    plt.subplot(2, 1, 1)
    idxs=potentialTxs.loc[:,['y', 'x']].sum(axis=1)!=0
    if sum(idxs)!=0:        
        plt.scatter(potentialTxs.loc[idxs,'frame']-5, np.sum(potentialTxs.loc[idxs,['y', 'x']], axis=1), color='black', label='coordinates found')
        plt.ylim([50,np.max(np.sum(potentialTxs.loc[idxs,['y', 'x']], axis=1))*1.2])
    plt.scatter(particle['frame'], np.sum(particle.loc[:,['y', 'x']], axis=1), label='coordinates from HCT')
    plt.legend(loc=0)
    plt.subplot(2, 1, 2)
    plt.plot(potentialTxs['frame'], potentialTxs['mrna'])
    plt.savefig(plotFileName)
    
def cleanClusters(clsFrm,Tx_mask_new):
    for i in range(len(clsFrm)):
        if clsFrm[i].size!=0:  
            mask_in_frames = Tx_mask_new[clsFrm[i][:,1], clsFrm[i][:,2]]
            clsFrm[i] = clsFrm[i][mask_in_frames]

    if len(clsFrm[findFitstTxFrame(clsFrm)])>= 2:
        print('choose spot!')
        brightestSpot = clsFrm[findFitstTxFrame(clsFrm)][np.argmax(findSpotBrightness(clsFrm[findFitstTxFrame(clsFrm)], sequenceCell[findFitstTxFrame(clsFrm)]))]
        clsFrm[findFitstTxFrame(clsFrm)] = brightestSpot.reshape(1,5)
        print(brightestSpot)
    return clsFrm

In [6]:
def saveTableAndCoordinates(potentialTxsFinal):
    columnsid=['frame']
    for clidu in np.unique(potentialTxsFinal.cluster_id):
        if clidu ==0:
            Tx1 = potentialTxsFinal[potentialTxsFinal['cluster_id']==0]
            tableToDisplay = pd.DataFrame(Tx1.loc[:,['frame', 'mrna']].astype(int))
            columnsid.append('mrna_'+str(clidu))
        else:
            Tx2 = potentialTxsFinal[potentialTxsFinal['cluster_id']==clidu]
            tableToDisplay = pd.concat([tableToDisplay , Tx2.loc[:,['mrna']].astype(int)], axis=1)
            columnsid.append('mrna_'+str(clidu))
    tableToDisplay.columns = columnsid
    potentialTxsFinal.to_pickle(pathToResults+'/'+cellNumber+'_mrna_coordinates.pkl')
    tableToDisplay.to_csv(pathToResults+'/'+cellNumber+'_mrna.csv')


## Select home folder

In [7]:
midentifier = 'cell_'

In [8]:
baseFolder = choose_home_folder() 
thresholdFile = pd.read_csv(baseFolder+'/thresholds.csv', index_col=0)


print("Chosen home folder:", baseFolder) # folder containg folder of movies
sessionNames = [os.path.join(baseFolder+'/'+i) for i in os.listdir(baseFolder) if midentifier in i and os.path.isdir(os.path.join(baseFolder,i))]

Chosen home folder: C:/Users/uid-1204/Desktop/bigFIS_trial_data_june2025/Hela_h9_h2_o1p1_mcp_td_SG_1hrbasal_14hr_10ng_TNF_75ms_5%_2_F00


In [10]:
pathToResults= baseFolder+'/results_new_new/'


## Enter the alpha chosen in the last step

In [11]:
al = input('Enter the alpha chosen in the last step :')

Enter the alpha chosen in the last step :0.6


## Choose the movie

In [12]:
# Create the root window
root = Tk()
root.geometry('180x200')
 
# Create a listbox
listbox = Listbox(root, width=90, height=100, selectmode=SINGLE)
 
# Inserting the listbox items
for i in range(len(sessionNames)):
    listbox.insert(i+1, sessionNames[i].split('/')[-1])

# Function for printing the
# selected listbox value(s)
def selected_item():
    global chosenCell
    # Traverse the tuple returned by
    # curselection method and print
    # corresponding value(s) in the listbox
    chosenCell = listbox.get(listbox.curselection())
    root.destroy()
    
# Create a button widget and
# map the command parameter to
# selected_item function
btn = Button(root, text='Choose', command=selected_item)
 
# Placing the button and listbox
btn.pack(side='bottom')
listbox.pack()
 
root.mainloop()

## Specify Input and Output folders, identifiers

In [14]:
cellNumber = chosenCell
print('cellNumber, ', cellNumber)

trackFilePath = os.path.join(baseFolder,cellNumber)
trackFilesToAnalyse = [os.path.join(trackFilePath,i) for i in os.listdir(trackFilePath) if 'mrnaTracks' in i]
if len(trackFilesToAnalyse)!=0:
    trackPath = Path(trackFilesToAnalyse[0])

    trackFile = trackPath.name
    cellFolderPath = trackPath.parents[0]


    thresholdFile = pd.read_csv(cellFolderPath.parent.joinpath('thresholds.csv'), index_col=0)
    extension = '.tif'
    print('cellNumber = ', cellNumber)
    idx = np.where(thresholdFile['cell'] == cellNumber)[0][0]

    thresholdSelected = thresholdFile['threshold'][idx]
    sequenceCell = imr(str(cellFolderPath.joinpath('*.tif')))
    mipSequenceCell = np.max(sequenceCell, axis=1)
    MaxTimePoint = sequenceCell.shape[0]
    spcl = np.load(cellFolderPath.joinpath(str(cellNumber)+'_spots_and_clusters_27_May'+al+'.npz'),allow_pickle=True)
    reference_spot = spcl['reference_spot']
    spotsFrame = spcl['spotsFrame']
    newClusterFile = np.load(os.path.join(pathToResults,(str(cellNumber)+'newClusters.npz')), allow_pickle=True)
    newClusterFrame = newClusterFile['newClusterFrame']

cellNumber,  cell_17
cellNumber =  cell_17


In [15]:
potentialTxsFinal = pd.read_pickle(os.path.join(pathToResults,cellNumber+'_finalDataframe.pkl'))
dftracksFinal = columnHd, newClusterFrame, tableToDisplay, dftracksFinal = makeTrackData(potentialTxsFinal)

## Display Results ( Run after analysing all Transcription sites)

In [16]:

cell_label_clean_file = cellFolderPath.parent.joinpath('masks_tx/blur_'+cellNumber+'.npy')
cell_label_clean = np.load(cell_label_clean_file)
def getDetectedPointsForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-1])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:]
    return pts_coords

def getDetectedClustersForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-3])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:3]
    return pts_coords

def set_pts_features(pts_layer, cls_layer, pts_coordinates, cluster_coordinate, image_layer_ndim, step): #TxLayer
    # step is a 4D coordinate with the current slider position for each dim
    frameNumber = step[0]  # grab the leading ("time") coordinate
    if image_layer_ndim==4:
        pts_layer.data = pts_coordinates[frameNumber]
        cls_layer.data = cluster_coordinate[frameNumber][:,:3]
    else:
        pts_layer.data = getDetectedPointsForFrame(pts_coordinates,frameNumber)
        cls_layer.data = getDetectedClustersForFrame(cluster_coordinate,frameNumber)


selectedThreshold = int(thresholdSelected)
pts_coordinates = spotsFrame
cluster_coordinate = newClusterFrame
viewer = napari.Viewer()
image_layer = viewer.add_image(
        mipSequenceCell, colormap='green' #maxImageCell
        )
if image_layer.data.ndim == 4:
    bigfish_Spots = viewer.add_points(
            spotsFrame[int(image_layer.data.shape[0]/2-1)],
            face_color='#00000000',
            size=4,
            edge_width=0.3,
            edge_width_is_relative=False,
            edge_color='white',
            name = 'bigFish Detected Spots'
            )
    bigfish_clusters = viewer.add_points(
        cluster_coordinate[int(image_layer.data.shape[0]/2-1)][:,:3],
        face_color='#00000000',
        size=8,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='red',
        symbol='diamond',
        name = 'bigFish Clusters'
        )

elif image_layer.data.ndim == 3:
    bigfish_Spots = viewer.add_points(
            getDetectedPointsForFrame(pts_coordinates,int(image_layer.data.shape[0]/2-1)),
            face_color='#00000000',
            size=4,
            edge_width=0.3,
            edge_width_is_relative=False,
            edge_color='white',
            name = 'bigFish Detected Spots'
            )
    bigfish_clusters = viewer.add_points(
        getDetectedClustersForFrame(cluster_coordinate,int(image_layer.data.shape[0]/2-1)),
        face_color='#00000000',
        size=8,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='red',
        symbol='diamond',
        name = 'bigFish Clusters'
        )

bigfish_tracks = viewer.add_tracks(dftracksFinal, 
                                   name='TS tracks')

viewer.dims.events.current_step.connect(
        lambda event: set_pts_features(bigfish_Spots, bigfish_clusters, pts_coordinates, cluster_coordinate,image_layer.data.ndim, event.value)
        )

@magicgui(call_button='Add Transcription site',main_window=False,
          saveTableButton=dict(widget_type="PushButton", text="Save mrna data"),
          loadMaskBtn=dict(widget_type="PushButton", text="Load Mask")
         )
def onAddTranscriptionSite(saveTableButton=True,
                           add_to_id=1,
                           cluster_threshold=1,
                           cluster_Gamma=4,
                          loadMaskBtn=True):
    print(viewer.dims.current_step[0],'\n',spotsFrame[viewer.dims.current_step[0]][list(viewer.layers[1].selected_data)[0]])
    newtxs = findMissingTxSite(selectedThreshold, sequenceCell, reference_spot, cellNumber, label(cell_label_clean).astype(bool), viewer.dims.current_step[0], spotsFrame[viewer.dims.current_step[0]][list(viewer.layers[1].selected_data)[0]], cluster_threshold, cluster_Gamma, 1, 600, (600,121,121), (600,150,150), False)
    if newtxs.size!=0:
        print('new tx found', newtxs)
        potentialTxsFinal.loc[np.logical_and(potentialTxsFinal['cluster_id']==add_to_id,potentialTxsFinal['frame']==viewer.dims.current_step[0]),['z','y','x','mrna']]=newtxs[:,:-1]
        columnHd, newClusterFrame, tableToDisplay, dftracksFinal = makeTrackData(potentialTxsFinal)
        indexToAdd = np.where(np.logical_and(potentialTxsFinal['frame']==viewer.dims.current_step[0], potentialTxsFinal['cluster_id']==add_to_id))[0]
        
        save_val = {'data': newtxs[0][3], 'row': viewer.dims.current_step[0], 'column': add_to_id+1, 'column_header': columnHd[add_to_id], 'row_header': str(viewer.dims.current_step[0])}
        table.data[viewer.dims.current_step[0], add_to_id+1]=newtxs[0][3]
        catchval(save_val)
        newtxs[:,4] = add_to_id
        potentialTxsFinal.to_pickle(pathToResults+'/'+cellNumber+'_mrna_coordinates.pkl') 
        
@onAddTranscriptionSite.saveTableButton.clicked.connect
def catchval(new_value):
    if new_value==False:
        table.to_dataframe().to_csv(baseFolder+'/'+cellNumber+'/'+cellNumber+'_mrna.csv')
    else:  
        print('new value saved')
        mrnaTable[new_value['column_header']][new_value['row_header']] = new_value['data']
        table.to_dataframe().to_csv(pathToResults+'/'+cellNumber+'_mrna.csv')
        

@onAddTranscriptionSite.loadMaskBtn.clicked.connect
def onLoadMask():
    print('maskloaded')
    viewer.add_image(label(cell_label_clean))
    
tablePath = pathToResults+'/'+cellNumber+'_mrna.csv'
if os.path.exists(tablePath):
    dft = pd.read_csv(tablePath, index_col=0)
    mrnaTable = dft.astype(int).to_dict()
else:
    mrnaTable = tableToDisplay.astype(int).to_dict()
tableToDisplay.to_csv(pathToResults+'/'+cellNumber+'_mrna.csv')
potentialTxsFinal.to_pickle(pathToResults+'/'+cellNumber+'_mrna_coordinates.pkl')

table = Table(value=mrnaTable)
table.changed.connect(catchval)
tablewidget = viewer.window.add_dock_widget(table, tabify=False, name = 'mrna table')
viewer.window.add_dock_widget(onAddTranscriptionSite, name = 'mrna table')

33 
 [  1 104  66]
done!
new tx found [[  1. 104.  66.   2.   0.]]
new value saved
new value saved


In [36]:
dftracksFinal

,cluster_id,frame,y,x
359,0,359,20.0,49.0
360,0,360,21.0,49.0
361,0,361,20.0,48.0
362,0,362,20.0,46.0
363,0,363,21.0,48.0
364,0,364,22.0,49.0
365,0,365,21.0,50.0
629,1,629,28.0,67.0
630,1,630,28.0,68.0
631,1,631,28.0,68.0


In [28]:
txid=1
meanxCoordinate = np.mean(potentialTxsFinal.loc[np.logical_and(potentialTxsFinal['cluster_id']==txid,potentialTxsFinal['x']!=0) ,'x'].values)
sdxCoordinate = np.std(potentialTxsFinal.loc[np.logical_and(potentialTxsFinal['cluster_id']==txid,potentialTxsFinal['x']!=0) ,'x'].values)

meanyCoordinate = np.mean(potentialTxsFinal.loc[np.logical_and(potentialTxsFinal['cluster_id']==txid,potentialTxsFinal['y']!=0) ,'y'].values)
sdyCoordinate = np.std(potentialTxsFinal.loc[np.logical_and(potentialTxsFinal['cluster_id']==txid,potentialTxsFinal['y']!=0) ,'y'].values)

meanzCoordinate = np.mean(potentialTxsFinal.loc[np.logical_and(potentialTxsFinal['cluster_id']==txid,potentialTxsFinal['z']!=0) ,'z'].values)
sdzCoordinate = np.std(potentialTxsFinal.loc[np.logical_and(potentialTxsFinal['cluster_id']==txid,potentialTxsFinal['z']!=0) ,'z'].values)

In [29]:
meanCoordinates = [meanzCoordinate, meanyCoordinate, meanxCoordinate]
sdCoordinates = [sdzCoordinate, sdyCoordinate, sdxCoordinate]

In [30]:
limitsz = [np.round(meanzCoordinate-1.5*sdzCoordinate), np.round(meanzCoordinate+1.5*sdzCoordinate)]
limitsy = [np.round(meanyCoordinate-1.5*sdyCoordinate), np.round(meanyCoordinate+1.5*sdyCoordinate)]
limitsx = [np.round(meanxCoordinate-1.5*sdxCoordinate), np.round(meanxCoordinate+1.5*sdxCoordinate)]

In [31]:
limitsy

[125.0, 132.0]

In [32]:
limitsx

[145.0, 152.0]

In [33]:
limitsz

[2.0, 4.0]

In [34]:
selectedThreshold

540

In [46]:
emptyFrames =[]
potentialSpots =[]
for i in range(MaxTimePoint):
    idxs = np.where(np.round(np.sum(np.abs(spotsFrame[i][:,0:] - meanCoordinates), axis=1))<=10)[0]
    if idxs.size!=0:
        emptyFrames.append(i)
        potentialSpots.append([i,spotsFrame[i][idxs,:]])
        print('frame '+str(i)+' = '+ str(spotsFrame[i][idxs[:],:])+' diff = '+ str(np.round(np.sum(np.abs(spotsFrame[i][idxs,0:] - meanCoordinates), axis=1))))

frame 148 = [[  5 130 150]] diff = [5.]
frame 203 = [[  3 131 148]] diff = [4.]
frame 204 = [[  3 132 147]] diff = [6.]
frame 205 = [[  3 132 149]] diff = [4.]
frame 206 = [[  3 134 146]] diff = [9.]
frame 207 = [[  3 131 148]] diff = [4.]
frame 208 = [[  3 132 148]] diff = [5.]
frame 209 = [[  3 132 149]] diff = [4.]
frame 211 = [[  3 130 149]] diff = [2.]
frame 212 = [[  2 130 148]] diff = [3.]
frame 213 = [[  5 132 152]] diff = [9.]
frame 227 = [[  8 132 148]] diff = [10.]
frame 251 = [[  3 129 150]] diff = [2.]
frame 252 = [[  3 132 150]] diff = [5.]
frame 253 = [[  3 128 149]] diff = [1.]
frame 254 = [[  3 132 149]] diff = [4.]
frame 255 = [[  3 132 150]] diff = [5.]
frame 256 = [[  3 130 151]] diff = [4.]
frame 257 = [[  3 131 151]] diff = [5.]
frame 258 = [[  3 131 152]] diff = [6.]
frame 259 = [[  2 131 144]
 [  3 133 150]] diff = [8. 6.]
frame 260 = [[  3 132 150]] diff = [5.]
frame 261 = [[  3 132 150]] diff = [5.]
frame 262 = [[  3 129 150]] diff = [2.]
frame 263 = [[  3 131

In [47]:
potentialSpots

[[148, array([[  5, 130, 150]])],
 [203, array([[  3, 131, 148]])],
 [204, array([[  3, 132, 147]])],
 [205, array([[  3, 132, 149]])],
 [206, array([[  3, 134, 146]])],
 [207, array([[  3, 131, 148]])],
 [208, array([[  3, 132, 148]])],
 [209, array([[  3, 132, 149]])],
 [211, array([[  3, 130, 149]])],
 [212, array([[  2, 130, 148]])],
 [213, array([[  5, 132, 152]])],
 [227, array([[  8, 132, 148]])],
 [251, array([[  3, 129, 150]])],
 [252, array([[  3, 132, 150]])],
 [253, array([[  3, 128, 149]])],
 [254, array([[  3, 132, 149]])],
 [255, array([[  3, 132, 150]])],
 [256, array([[  3, 130, 151]])],
 [257, array([[  3, 131, 151]])],
 [258, array([[  3, 131, 152]])],
 [259,
  array([[  2, 131, 144],
         [  3, 133, 150]])],
 [260, array([[  3, 132, 150]])],
 [261, array([[  3, 132, 150]])],
 [262, array([[  3, 129, 150]])],
 [263, array([[  3, 131, 151]])],
 [264, array([[  3, 130, 148]])],
 [265, array([[  3, 130, 149]])],
 [266, array([[  3, 128, 150]])],
 [267, array([[  3, 

In [48]:
for j in range(len(potentialSpots)):
    timepoint= potentialSpots[j][0]
    spotTemp = potentialSpots[j][1]
    if len(spotTemp)>1:
        print('Choose spot for frame '+str(timepoint))
        nw1 = findTxSite(selectedThreshold, sequenceCell, reference_spot, cellNumber, timepoint, label(cell_label_clean).astype(bool), spotTemp[0], 1, 3, 1, reorderTF = False)
        nw2 = findTxSite(selectedThreshold, sequenceCell, reference_spot, cellNumber, timepoint, label(cell_label_clean).astype(bool), spotTemp[1], 1, 3, 1, reorderTF = False)
        choosenspot = input(print('\nEnter 0 for spot '+str(nw1)+str('\nEnter 1 for spot ')+str(nw2)))
        #check within limits before choosing
        if choosenspot==0:
            if checkWithinLimits(nw1[0][:3],  limitsz, limitsy, limitsx):
                print('Not within limits')
            else:
                newtxs=nw1
                print('new tx found at frame '+str(timepoint), newtxs)
        elif choosenspot==1:
            if checkWithinLimits(nw2[0][:3],  limitsz, limitsy, limitsx):
                print('Not within limits')
            else:
                newtxs=nw2
                print('new tx found at frame '+str(timepoint), newtxs)
    elif len(spotTemp)==1:       
        newtxs = findTxSite(selectedThreshold, sequenceCell, reference_spot, cellNumber, timepoint, label(cell_label_clean).astype(bool), spotTemp, 1, 3, 1, reorderTF = False)
    if newtxs.size!=0:
        print('new tx found at frame '+str(timepoint), newtxs)
        potentialTxsFinal.loc[np.logical_and(potentialTxsFinal['cluster_id']==txid,potentialTxsFinal['frame']==timepoint),['z','y','x','mrna']]=newtxs[:,:-1]


done!
new tx found at frame 148 [[  5. 130. 150.   1.   0.]]
done!
new tx found at frame 203 [[  3. 131. 148.   1.   0.]]
done!
new tx found at frame 204 [[  3. 132. 147.   1.   0.]]
done!
new tx found at frame 205 [[  3. 132. 149.   1.   0.]]
done!
not found (not in mask)
done!
new tx found at frame 207 [[  3. 131. 148.   1.   0.]]
done!
new tx found at frame 208 [[  3. 132. 148.   1.   0.]]
done!
new tx found at frame 209 [[  3. 132. 149.   1.   0.]]
done!
new tx found at frame 211 [[  3. 130. 149.   1.   0.]]
done!
new tx found at frame 212 [[  2. 130. 148.   1.   0.]]
done!
new tx found at frame 213 [[  5. 132. 151.   2.   0.]]
done!
new tx found at frame 227 [[  8. 132. 148.   1.   0.]]
done!
new tx found at frame 251 [[  3. 129. 150.   1.   0.]]
done!
new tx found at frame 252 [[  3. 132. 150.   2.   0.]]
done!
new tx found at frame 253 [[  3. 128. 149.   2.   0.]]
done!
new tx found at frame 254 [[  3. 131. 149.   3.   0.]]
done!
not found (not close to last location)
done!
new 

done!
new tx found at frame 795 [[  4. 121. 148.   1.   0.]]
done!
new tx found at frame 796 [[  4. 121. 149.   1.   0.]]
done!
new tx found at frame 797 [[  4. 121. 149.   1.   0.]]
done!
new tx found at frame 800 [[  4. 120. 149.   1.   0.]]
done!
new tx found at frame 802 [[  4. 121. 148.   1.   0.]]
done!
new tx found at frame 803 [[  4. 121. 149.   1.   0.]]
done!
new tx found at frame 806 [[  4. 121. 148.   1.   0.]]
done!
new tx found at frame 808 [[  4. 121. 149.   1.   0.]]
done!
new tx found at frame 810 [[  5. 129. 145.   1.   0.]]
done!
new tx found at frame 811 [[  5. 128. 145.   2.   0.]]


In [38]:
def findTxSite(selectedThreshold, sequenceCell, reference_spot,cellNumber, t,Tx_label_clean, lastLocation, bETA, gAMA=5, numCs=2, reorderTF = False):
    spots_T, clusters_T, _ = getSpotAndClusters(sequenceCell,
                                             reference_spot, 
                                             cellnumber=cellNumber, 
                                             startTime=t,
                                             stopTime=t+1, 
                                             thresholdManual=selectedThreshold, 
                                             beta=bETA, 
                                             gamma=gAMA,
                                             numberOfSpots=numCs,
                                             radiusCluster=400, 
                                             voxelSize=voxelRadius, 
                                             objectSize=objectRadius,
                                             reorder=reorderTF,
                                             extensionMov='.tif',
                                             showProgress=False)

    if len(clusters_T[0])>3*len(spots_T[0]):
        print(len(spots_T[0]))
        print(len(clusters_T[0]))
        print('beta very low! Increase the beta!')
        
    mask_in_frames = Tx_label_clean[clusters_T[0][:,1], clusters_T[0][:,2]]
    missingTx0 = clusters_T[0][mask_in_frames]
    if missingTx0.size!=0:
        idxs = np.where(np.sum(np.abs(missingTx0[:,0:3] - lastLocation), axis=1)<=1)[0]
        if idxs.size!=0:
            missingTx = mergeTxSites(missingTx0[idxs,:])
            return missingTx
        else:
            print('not found (not close to last location)')
            return missingTx0[idxs,:]
    else:
        print('not found (not in mask)')
        return missingTx0

In [39]:
def checkWithinLimits(point, limz, limy, limx):
    if point[0]<=limz[1] and point[0]>=limz[0]:
        zz=1
    else:
        zz=0
    if point[1]<=limy[1] and point[1]>=limy[0]:
        yy=1
    else:
        yy=0
    if point[2]<=limx[1] and point[2]>=limz[0]:
        xx=1
    else:
        xx=0
    if zz+yy+xx==3:
        return True
    else:
        return False
    

In [40]:
def checkClosestPos()

SyntaxError: invalid syntax (1585477363.py, line 1)

In [52]:
newtxs.size

5

  File "/home/rachel/anaconda3/envs/bigfishLive/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/rachel/anaconda3/envs/bigfishLive/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/rachel/anaconda3/envs/bigfishLive/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/rachel/anaconda3/envs/bigfishLive/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/rachel/anaconda3/envs/bigfishLive/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/rachel/anaconda3/envs/bigfishLive/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/home/rachel/anaconda3/envs/bigfishLive/lib/python3.8/asyncio/base_events.py", line 570, in run_forever